### Assignment 4 - Question 4

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
def downsample(img, scale_percent=50):
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    return cv2.resize(img, (width, height))

img1 = cv2.imread("./University_College,_University_of_Toronto.jpg")
img2 = cv2.imread("./University_College_Lawn,_University_of_Toronto,_Canada.jpg")
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
img1 = downsample(img1, 70)
img2 = downsample(img2, 70)
print(img1.shape)
print(img2.shape)

#### 4.1: Calculate eigenvalues of second moment matrix 

In [ ]:
def second_moment_matrix(img, sigma):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray,(5,5),7)

    Ix = cv2.Sobel(blur, cv2.CV_64F, 1, 0, ksize=5)
    Iy = cv2.Sobel(blur, cv2.CV_64F, 0, 1, ksize=5)

    IxIy = np.multiply(Ix, Iy)
    Ix2 = np.multiply(Ix, Ix)
    Iy2 = np.multiply(Iy, Iy)

    Ix2_blur = cv2.GaussianBlur(Ix2,(7,7),sigma) 
    Iy2_blur = cv2.GaussianBlur(Iy2,(7,7),sigma) 
    IxIy_blur = cv2.GaussianBlur(IxIy,(7,7),sigma)
    
    m, n, _ = img.shape
    e = np.zeros((m, n, 2))
    for i in range(m):
        for j in range(n):
            e[i][j] = np.linalg.eigvals(np.array([[Ix2_blur[i][j], IxIy_blur[i][j]],
                                                  [IxIy_blur[i][j], Iy2_blur[i][j]]]))
    return e

e1 = second_moment_matrix(img1, 2)
e2 = second_moment_matrix(img2, 2)

#### 4.2: Plot eigenvalues

In [ ]:
fig, axes = plt.subplots(1, 2, figsize = (15, 5))

axes[0].set_title("Scatter Plot of Eigenvalues: image 1")
axes[0].scatter(e1[:,:,0], e1[:,:,1], s=0.2)
axes[1].set_title("Scatter Plot of Eigenvalues: image 2")
axes[1].scatter(e1[:,:,0], e1[:,:,1], s=0.2)


#### 4.3: Pick thresholds and detect corners

In [ ]:
def detect_corners(img, e, threshold):
    img_copy = img.copy()
    R = np.minimum(e[:,:,0], e[:,:,1])
    img_copy[R > threshold]=[255, 255, 0] 
    plt.figure(figsize=(7,7))
    plt.imshow(img_copy)


Image 1, pick threshold of 6e5.

In [ ]:
detect_corners(img1, e1, 6e5)

Image 2, pick threshold of 8e5.

In [ ]:
detect_corners(img2, e2, 5e5)

#### 4.4: Different Gaussian kernel

Instead of sigma=2, this time we use sigma of 30.

In [ ]:
img1_copy = img1.copy()
img2_copy = img2.copy()

e1_ = second_moment_matrix(img1_copy, 30)
e2_ = second_moment_matrix(img2_copy, 30)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize = (15, 5))

axes[0].set_title("Scatter Plot of Eigenvalues: image 1")
axes[0].scatter(e1_[:,:,0], e1_[:,:,1], s=0.2)
axes[1].set_title("Scatter Plot of Eigenvalues: image 2")
axes[1].scatter(e2_[:,:,0], e2_[:,:,1], s=0.2)

In [ ]:
detect_corners(img1_copy, e1_, 6e5)

In [ ]:
detect_corners(img2_copy, e2_, 5e5)

Explain how this choice influenced the corner detection in each of the images

##### Comments:
- Change sigma to 30, which is 15 times of its original value of 2.
- For each image, we observe more corners are detected and detection areas are larger.
- Eigenvalues in the scatter plot are more spread out with this larger sigma value. 
- Since eigenvalues determine whether a region is an edge, a corner or flat, with a larger sigma, more value points are classified as "corner", using the same threshold.